In [1]:
import os
import pandas as pd
pd.options.display.max_rows = 10
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import lightgbm as lightgbm
import xgboost as xgb

start = time.time()
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_features = pd.read_csv('test_features.csv')
test_labels = pd.read_csv('test_labels.csv')

#pip = Pipeline([('std_scaler', StandardScaler())])
pip = Pipeline([('minmax_scaler', MinMaxScaler())])
featureNames = train_features.columns
train_features = pip.fit_transform(train_features)
cv = 5

display(train_labels)
display(train_features)

,hosp_inz_entries,death_inz_entries,testPositvity,googleMobility
0,0.021429,0.000000,0.014211,5.690476
1,0.062857,0.000000,0.032002,3.857143
2,0.062857,0.000000,0.044182,3.714286
3,0.104286,0.000000,0.048949,1.619048
4,0.104286,0.000000,0.037903,2.142857
...,...,...,...,...
9264,0.768571,0.090000,0.046355,-22.166667
9265,0.667143,0.098571,0.046011,-22.166667
9266,0.611429,0.071429,0.045847,-22.166667
9267,0.564286,0.080000,0.044938,-22.166667


array([[0.00000000e+00, 0.00000000e+00, 2.08947313e-04, ...,
        2.26361032e-01, 8.79343851e-02, 2.54783619e-01],
       [0.00000000e+00, 0.00000000e+00, 2.78596418e-04, ...,
        2.26361032e-01, 8.79343851e-02, 2.54783619e-01],
       [0.00000000e+00, 0.00000000e+00, 3.48245522e-04, ...,
        2.26361032e-01, 8.79343851e-02, 2.54783619e-01],
       ...,
       [4.95003152e-01, 2.17957897e-01, 1.03868240e-01, ...,
        4.81375358e-01, 1.70477125e-01, 4.49779835e-01],
       [4.94292062e-01, 2.18828778e-01, 1.05479079e-01, ...,
        4.81375358e-01, 1.70477125e-01, 4.49779835e-01],
       [4.88904114e-01, 2.18552749e-01, 1.06801199e-01, ...,
        4.81375358e-01, 1.70477125e-01, 4.49779835e-01]])

In [2]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # SVR
    display("---SVR---"+label+"---")
    start = time.time()
    parameters = {'C':[0.1]}
    reg = GridSearchCV(svm.SVR(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---SVR---hosp_inz_entries---'

SVR(C=0.1)

'Scores:[[0.05950023 0.07453651 0.06076929 0.09227579 0.05269996]]'

'Mean:0.06795635703952972'

'Standard deviation:0.014075872157579324'

...

SVR(C=0.1)

'Scores:[[0.06614339 0.08356044 0.05633843 0.06165636 0.06348308]]'

'Mean:0.06623634092697808'

'Standard deviation:0.009237515163533489'

'----------End of evaluating (1.6380016803741455)----------'

In [3]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # Ridge regression
    display("---Ridge---"+label+"---")
    start = time.time()
    parameters = {'alpha':[0.1, 1, 10]}
    reg = GridSearchCV(linear_model.Ridge(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    #display(pd.DataFrame(reg.best_estimator_.coef_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---Ridge---hosp_inz_entries---'

Ridge(alpha=0.1)

'Scores:[[0.03087904 0.03111692 0.02429828 0.03369723 0.02388065]]'

'Mean:0.028774421120265337'

'Standard deviation:0.003953115685576749'

...

Ridge(alpha=1)

'Scores:[[0.03310734 0.03277716 0.0242469  0.02284923 0.03004456]]'

'Mean:0.028605037642457958'

'Standard deviation:0.004286542956571969'

'----------End of evaluating (0.5124070644378662)----------'

In [4]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # Lasso regression
    display("---Lasso---"+label+"---")
    start = time.time()
    parameters = {'alpha':[0.00001, 0.0001]}
    reg = GridSearchCV(linear_model.Lasso(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.coef_.transpose(), index=featureNames, columns=['coefficients']).sort_values(['coefficients'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---Lasso---hosp_inz_entries---'

Lasso(alpha=0.0001)

'Scores:[[0.02664086 0.02515486 0.02356835 0.02719636 0.01745393]]'

'Mean:0.024002871285291914'

'Standard deviation:0.0035088985543820232'

...

'Scores:[[0.0292514  0.03101672 0.02171582 0.02032745 0.02530927]]'

'Mean:0.025524133438071077'

'Standard deviation:0.004138019072258814'

,coefficients
Death inz_entries female,0.507942
Death inz_entries male,0.502378
Cases inz_entries 80+,0.082079
Death inz_entries 80+,0.073473
hosp_inz_entries,0.036926
...,...
clouds_day_0,-0.001823
Cases inzsumTotal 80+,-0.002358
Death inzsumTotal 80+,-0.006293
maxNeighborIncidence,-0.006640


'----------End of evaluating (5.103142499923706)----------'

In [ ]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # MLP regression
    display("---MLP---"+label+"---")
    start = time.time()
    parameters = {'hidden_layer_sizes':[(100,50,50,20,10,10)]}
    reg = GridSearchCV(MLPRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

In [ ]:
for label in ['death_inz_entries','hosp_inz_entries', 'testPositvity','googleMobility']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # RandomForestRegressor
    display("---RandomForestRegressor---"+label+"---")
    start = time.time()
    parameters = {'n_estimators':[20]}
    reg = GridSearchCV(RandomForestRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.feature_importances_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))


In [ ]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()
    
    # lightGBM
    display("---lightGBM---"+label+"---")
    start = time.time()
    parameters = {'n_estimators':[100]}
    reg = GridSearchCV(lightgbm.LGBMRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.feature_importances_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

In [ ]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()
    
    # XGBoost
    display("---XGBoost---"+label+"---")
    start = time.time()
    parameters = {'n_estimators':[10], 'reg_alpha': [0], 'reg_lambda': [1]}
    reg = GridSearchCV(xgb.XGBRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    #display(reg.best_estimator_.get_xgb_params())
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.feature_importances_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

In [ ]:
# PCA
from sklearn.decomposition import IncrementalPCA

ipca = IncrementalPCA()
ipca.fit(train_features)
display(pd.DataFrame(ipca.singular_values_.transpose(), index=featureNames, columns=['singular value']).sort_values(['singular value'], ascending=False))